In [65]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold

установка библиотек и чтение файла

In [66]:
df = pd.read_csv("vgsales_2_1.csv")
df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,12200,59.0,6.0,2004.0,Sports,6.0,5.0,0.01,0.00,0.00,0.07
1,10814,261.0,21.0,2002.0,Action,6.0,7.0,0.02,0.00,0.00,0.10
2,894,242.0,18.0,2008.0,Sports,89.0,91.0,0.00,0.29,0.12,1.93
3,4808,510.0,11.0,2008.0,Action,117.0,0.0,0.00,0.40,0.00,0.40
4,13925,104.0,22.0,2015.0,Sports,120.0,0.0,0.04,0.00,0.00,0.04
...,...,...,...,...,...,...,...,...,...,...,...
548,4143,396.0,18.0,2008.0,Simulation,37.0,37.0,0.03,0.04,0.04,0.48
549,14462,146.0,21.0,2003.0,Racing,85.0,2.0,0.01,0.00,0.00,0.03
550,7513,147.0,1.0,2011.0,Action,68.0,16.0,0.03,0.00,0.01,0.20
551,872,474.0,20.0,2006.0,Shooter,122.0,82.0,0.65,0.02,0.20,1.96


Далее категориальные значения преобразовываем в бинарные, используем OrdinalEncoder, но Genre через OneHotEncoder, 
из-за малого количества уникальных данных. Через fit_transform преобразовываем старые данные в новые.

In [67]:
OrdEnc = OrdinalEncoder()
df[['Name', 'Platform', 'Year', 'Publisher']] = OrdEnc.fit_transform(df[['Name', 'Platform', 'Year', 'Publisher']])
OneHEnc = OneHotEncoder()
df['Genre'] = OneHEnc.fit_transform(df[['Genre']]).toarray()
df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,12200,59.0,6.0,19.0,0.0,6.0,5.0,0.01,0.00,0.00,0.07
1,10814,261.0,21.0,17.0,1.0,6.0,7.0,0.02,0.00,0.00,0.10
2,894,242.0,18.0,23.0,0.0,89.0,91.0,0.00,0.29,0.12,1.93
3,4808,510.0,11.0,23.0,1.0,117.0,0.0,0.00,0.40,0.00,0.40
4,13925,104.0,22.0,30.0,0.0,120.0,0.0,0.04,0.00,0.00,0.04
...,...,...,...,...,...,...,...,...,...,...,...
548,4143,396.0,18.0,23.0,0.0,37.0,37.0,0.03,0.04,0.04,0.48
549,14462,146.0,21.0,18.0,0.0,85.0,2.0,0.01,0.00,0.00,0.03
550,7513,147.0,1.0,26.0,1.0,68.0,16.0,0.03,0.00,0.01,0.20
551,872,474.0,20.0,21.0,0.0,122.0,82.0,0.65,0.02,0.20,1.96


Кросс-валидация является важной процедурой в области машинного обучения для оценки производительности моделей на новых данных. Она позволяет получить более объективную оценку способности модели обобщать и делать прогнозы на неизвестных данных.

K-блочная кросс-валидация (k-fold cross-validation) - один из распространенных вариантов кросс-валидации. Она разбивает исходный набор данных на k блоков (или фолдов) одинакового размера. Затем модель обучается и оценивается k раз, каждый раз используя один из блоков в качестве проверочного набора данных, а оставшиеся блоки - в качестве тренировочного набора данных. Таким образом, каждый блок используется в качестве проверочного набора данных ровно один раз.

n_splits=5: Определяет количество блоков (фолдов), на которые разбивается исходный набор данных. В данном случае используется 5 блоков.

shuffle=False: Определяет, должны ли данные быть перемешаны перед разделением на блоки. В данном случае данные не перемешиваются, то есть используется исходный порядок данных при разбиении.

random_state=None: Этот параметр используется только в том случае, если shuffle=True. Он контролирует порядок индексов, используемых для перемешивания данных. Если random_state имеет определенное числовое значение, то при каждом запуске процедуры кросс-валидации будет использоваться одно и то же случайное перемешивание данных. Если random_state=None, то порядок индексов будет случайным для каждого запуска.

In [68]:
Y = df["Genre"]
X = df.drop(columns = "Genre")

kf = KFold(n_splits=5, shuffle=False, random_state=None)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

разделение строки на список подстрок по разделителю.

In [69]:
kf.split(df)

<generator object _BaseKFold.split at 0x000001D386B0D120>

подключаюемся и объявляем переменные для обучения моделей( ближайших соседей, метода опорных векторов, дерева решений )

In [70]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
from sklearn.svm import SVC
svc = SVC()

используется функция train_test_split для выполнения k-кратной перекрестной валидации на наборе данных, обучения, предсказания различных варинтов моделей.

In [71]:
for train_index, test_index in kf.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_kfold = X.iloc[train_index]
    X_test_kfold = X.iloc[test_index]
    Y_train_kfold = Y.iloc[train_index]
    Y_test_kfold = Y.iloc[test_index]

    knc.fit(X_train_kfold,Y_train_kfold)
    predict_knc = knc.predict(X_test_kfold)

    dtc = dtc.fit(X_train_kfold,Y_train_kfold)
    predict_dtc = dtc.predict(X_test_kfold)

    svc.fit(X_train_kfold,Y_train_kfold)
    predict_svc = svc.predict(X_test_kfold)

TRAIN: [111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146
 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164
 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182
 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200
 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236
 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272
 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308
 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326
 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344
 345 346 347 348 349 350 351 352 353 354 355

вывод обученных моделей

In [72]:
print("Набор предсказаний дерева решений: ",predict_dtc)
print("Набор предсказаний К-ближайших соседей: ",predict_knc)
print("Набор предсказаний метода опорных векторов: ",predict_svc)

Набор предсказаний дерева решений:  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Набор предсказаний К-ближайших соседей:  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
Набор предсказаний метода опорных векторов:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

Регрессия – группа Моделей, используемых для прогнозирования непрерывных значений, таких как цены на недвижимость с учетом их характеристик. Для регресии будут использоваться тот же k-блочная кросс-валидация, как и было продемонстрировано выше для сравнения и выбора модели конкретной задачи прогнозного моделирования

In [73]:
Y = df["Global_Sales"]
X = df.drop(columns = "Global_Sales")

kf = KFold(n_splits=5, shuffle=False, random_state=None)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [74]:
kf.split(df)

<generator object _BaseKFold.split at 0x000001D386B0CA60>

переменные для регрессии опорных векторов, регрессии дерева решений и линейной регрессии.

In [75]:
from sklearn.linear_model import LinearRegression
lrc = LinearRegression()
from sklearn.svm import SVR
svr = SVR()
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()

In [76]:
for train_index, test_index in kf.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_kfold = X.iloc[train_index]
    X_test_kfold = X.iloc[test_index]
    Y_train_kfold = Y.iloc[train_index]
    Y_test_kfold = Y.iloc[test_index]

    lrc.fit(X_train_kfold,Y_train_kfold)
    predict_lrc = lrc.predict(X_test_kfold)

    svr.fit(X_train_kfold,Y_train_kfold)
    predict_svr = svr.predict(X_test_kfold)

    dtr = dtr.fit(X_train_kfold,Y_train_kfold)
    predict_dtr = dtr.predict(X_test_kfold)
    

TRAIN: [111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146
 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164
 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182
 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200
 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236
 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272
 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308
 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326
 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344
 345 346 347 348 349 350 351 352 353 354 355

вывод регресии моделей 

In [77]:
print("Набор предсказаний регрессии дерева решений: ",predict_dtr)
print("Набор предсказаний регрессии опорных векторов: ",predict_svr)
print("Набор предсказаний линейной регрессии: ",predict_lrc)

Набор предсказаний регрессии дерева решений:  [0.01 0.55 0.02 0.06 1.08 0.35 0.02 0.03 0.44 0.82 0.05 1.41 0.06 0.14
 0.52 0.19 8.84 1.41 0.36 0.04 1.25 0.26 0.81 0.17 2.86 0.57 0.02 0.01
 0.49 2.87 0.16 0.16 0.04 0.03 0.04 0.14 0.17 0.61 0.38 0.56 0.44 0.06
 0.87 0.7  0.29 0.73 0.01 0.13 0.02 1.54 0.01 0.4  0.49 0.04 0.04 0.26
 0.06 1.34 2.28 0.29 0.11 0.62 3.17 0.05 0.2  0.19 0.03 0.14 0.14 0.44
 0.04 0.08 0.04 0.05 0.29 0.52 0.67 0.47 1.01 1.   0.49 1.   0.08 0.73
 0.2  0.19 1.41 0.01 0.21 0.47 0.02 0.84 0.75 0.01 0.07 0.21 0.87 0.82
 0.09 1.   0.06 0.1  0.4  0.14 0.94 0.48 0.03 0.2  1.95 0.01]
Набор предсказаний регрессии опорных векторов:  [0.09347999 0.57883935 0.08293303 0.1531684  1.16414726 0.25558083
 0.08297307 0.08749179 0.41431389 0.93732327 0.12309514 1.43534707
 0.14127394 0.18093414 0.50212824 0.15839333 2.1465146  1.4411091
 0.25937077 0.10880975 1.33457255 0.16447114 0.89788482 0.16735218
 1.86282453 0.60628019 0.08313829 0.10550031 0.48029241 1.87933002
 0.17177533 0